In [1]:
#pandas for data manipulation
#numpy for computaion
#Sklearn - for Machine Learning model selection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#reading the new file
temp=pd.read_csv('trimmedData.csv')

In [3]:
X=temp['X']
X=X.values.reshape(-1,1)
Y=temp['Y']

# grouping of data 
and creating chunks of size window


In [4]:
#groupby Y value, (1,2,32,37,38)
gtemp=temp.groupby('Y')
#initializing y
y=[]

In [5]:
#dividing all on the basis of their label
#get_group(1) reutrns the group with y label 1
g1=gtemp.get_group(1) 
g2=gtemp.get_group(2)
g32=gtemp.get_group(32)
g37=gtemp.get_group(37)
g38=gtemp.get_group(38)

In [6]:
#n_features =1 (amplitude)
n_features=1
#initialise total data point to 0
totaldp=0
#window size =250 ==1 sec
window=250

#get amplitude values from X
X=g1['X']
#number of data points = number of rows in X / window
data_points=X.shape[0]//window
#appending y 
y=np.append(y,[1]*data_points)
#updating total datapoints
totaldp+=data_points
#removing left points
X=X.head(data_points*window)
#converting to array
X=np.array(X)
#reshaping it to data_points,window,n_features(n,250,1)
X=np.reshape(X,((data_points,window,n_features)))
#grouped X is the new amplitude chunk
groupedX=X

#same procedure for label 2
X=g2['X']
data_points=X.shape[0]//window
y=np.append(y,[2]*data_points)
totaldp+=data_points
X=X.head(data_points*window)
X=np.array(X)
X=np.reshape(X,((data_points,window,n_features)))
groupedX=np.append(groupedX,X)

#same procedure for label 32
X=g32['X']
data_points=X.shape[0]//window
y=np.append(y,[32]*data_points)
totaldp+=data_points
X=X.head(data_points*window)
X=np.array(X)
X=np.reshape(X,((data_points,window,n_features)))
groupedX=np.append(groupedX,X)


X=g37['X']
data_points=X.shape[0]//window
y=np.append(y,[37]*data_points)
totaldp+=data_points
X=X.head(data_points*window)
X=np.array(X)
X=np.reshape(X,((data_points,window,n_features)))
groupedX=np.append(groupedX,X)


X=g38['X']
data_points=X.shape[0]//window
y=np.append(y,[38]*data_points)
totaldp+=data_points
X=X.head(data_points*window)
X=np.array(X)
X=np.reshape(X,((data_points,window,n_features)))
groupedX=np.append(groupedX,X)


groupedX=np.reshape(groupedX,((totaldp,window,n_features)))

# extracting features

In [8]:
X=groupedX
#max value of colomn (axis specifis row or colomn)
a=np.max(X,axis=1)
#concatinate with minimum, second axis is for concatinate into colomn
a=np.concatenate((a,np.min(X,axis=1)),axis=1)
a=np.concatenate((a,np.average(X,axis=1)),axis=1)
#standard deviation
a=np.concatenate((a,np.std(X,axis=1)),axis=1)
#fft real part
a=np.concatenate((a,np.real(np.fft.rfftn(X,axes=(0,1)))[:,0]),axis=1)
X=a

In [9]:
Y

0          1
1          1
2          1
3          1
4          1
5          1
6          1
7          1
8          1
9          1
10         1
11         1
12         1
13         1
14         1
15         1
16         1
17         1
18         1
19         1
20         1
21         1
22         1
23         1
24         1
25         1
26         1
27         1
28         1
29         1
          ..
657402    37
657403    37
657404    37
657405    37
657406    37
657407    37
657408    37
657409    37
657410    37
657411    37
657412    37
657413    37
657414    37
657415    37
657416    37
657417    37
657418    37
657419    37
657420    37
657421    37
657422    37
657423    37
657424    37
657425    37
657426    37
657427    37
657428    37
657429    37
657430    37
657431    37
Name: Y, Length: 657432, dtype: int64

# algos

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(y_test.shape)

(868,)


# Data Classes
1 new mind wonder           
2 new sleepy        
32 peek experiences        
37 some meditation     
38 good meditation      

Mind Wonder and aleepy are 5-10 sec after it actually occoured

In [11]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [12]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
clf.score(X_test,y_test)
y_pre=clf.predict(X_test)

In [13]:
#Analysing Results - Accuracy, precision, recall, F1-score for each class
actual=y_test
predicted=y_pre
results = confusion_matrix(actual, predicted) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, predicted) )
#print ('Report : ')
print (classification_report(actual, predicted) )

Confusion Matrix :
[[368  21   0  22   3]
 [ 22  55   0   9   0]
 [  0   0   1   0   0]
 [ 28   5   0 218  16]
 [  3   0   0   6  91]]
Accuracy Score : 0.8444700460829493
              precision    recall  f1-score   support

         1.0       0.87      0.89      0.88       414
         2.0       0.68      0.64      0.66        86
        32.0       1.00      1.00      1.00         1
        37.0       0.85      0.82      0.84       267
        38.0       0.83      0.91      0.87       100

   micro avg       0.84      0.84      0.84       868
   macro avg       0.85      0.85      0.85       868
weighted avg       0.84      0.84      0.84       868



In [14]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred=clf.predict(X_test)

In [15]:
actual=y_test
predicted=y_pred
results = confusion_matrix(actual, predicted) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, predicted) )
#print ('Report : ')
print (classification_report(actual, predicted))

Confusion Matrix :
[[381  18   0  15   0]
 [ 25  61   0   0   0]
 [  0   0   1   0   0]
 [ 25   8   0 226   8]
 [  7   0   0   8  85]]
Accuracy Score : 0.868663594470046
              precision    recall  f1-score   support

         1.0       0.87      0.92      0.89       414
         2.0       0.70      0.71      0.71        86
        32.0       1.00      1.00      1.00         1
        37.0       0.91      0.85      0.88       267
        38.0       0.91      0.85      0.88       100

   micro avg       0.87      0.87      0.87       868
   macro avg       0.88      0.87      0.87       868
weighted avg       0.87      0.87      0.87       868



In [16]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)
y_pred=knn.predict(X_test)

In [17]:
actual=y_test
predicted=y_pred
results = confusion_matrix(actual, predicted) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, predicted) )
#print ('Report : ')
print (classification_report(actual, predicted) )

Confusion Matrix :
[[262  27   0  83  42]
 [ 62  18   0   6   0]
 [  1   0   0   0   0]
 [213   4   0  48   2]
 [ 73   0   0   6  21]]
Accuracy Score : 0.402073732718894
              precision    recall  f1-score   support

         1.0       0.43      0.63      0.51       414
         2.0       0.37      0.21      0.27        86
        32.0       0.00      0.00      0.00         1
        37.0       0.34      0.18      0.23       267
        38.0       0.32      0.21      0.25       100

   micro avg       0.40      0.40      0.40       868
   macro avg       0.29      0.25      0.25       868
weighted avg       0.38      0.40      0.37       868



C:\Users\Babbar\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Total number of chunks in each classa.      
1 -new mind wonder - 1260         
b. 2 -new sleepy - 261              
c. 32 -peak experience - 8        
d. 37 -some meditation - 785      
e. 38 -good meditation - 314    
2. Training by 67%(train data), Predicting the Test Data results and comparing to actual labels
3. Analysing Results - Accuracy, precision, recall, F1-score for each class in Random Forest Classifier with number of estimators =100 here the class with the worst accuracy is 2 - sleepy (Probably due to verysmall training set and high resemblance with class 1 - mind wonder(18 points shifted) this may be due to labeling the same way.
4. Best accuracy for class 1 (so if we increase the data we can expect the accuracy to go high)